# Chapter 97: Quantum Machine Learning

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand the basic principles of quantum computing and how they differ from classical computing.
- Identify potential applications of quantum machine learning (QML) to time‑series prediction problems.
- Explain key quantum algorithms relevant to machine learning: quantum support vector machines, quantum neural networks, and variational quantum eigensolvers.
- Recognise the current limitations of quantum hardware (noise, qubit count, coherence times) and their impact on practical QML.
- Implement a simple quantum classifier using Qiskit and apply it to a synthetic time‑series classification problem.
- Evaluate the potential advantages of QML for specific tasks (e.g., kernel methods, optimisation) and understand when classical approaches are still superior.
- Stay informed about the rapidly evolving landscape of quantum computing and its implications for time‑series forecasting.

---

## **97.1 Introduction to Quantum Machine Learning**

Quantum machine learning (QML) sits at the intersection of quantum computing and machine learning. It explores how quantum algorithms can be used to perform machine learning tasks more efficiently than classical algorithms, or how machine learning can be used to solve quantum problems.

For time‑series prediction, QML is still largely experimental. However, certain aspects of quantum computing suggest potential advantages:

- **Kernel methods**: Quantum computers can compute kernels that are hard to estimate classically, potentially improving support vector machines for complex datasets.
- **Optimisation**: Many machine learning problems involve optimisation (e.g., training neural networks). Quantum algorithms like the Quantum Approximate Optimisation Algorithm (QAOA) might offer speedups.
- **Sampling**: Quantum computers can sample from probability distributions that are difficult to model classically, which could benefit generative models.
- **Feature mapping**: Quantum feature maps can project data into exponentially high‑dimensional Hilbert spaces, which might make certain patterns linearly separable.

In this chapter, we will not build a production‑ready quantum model for NEPSE—quantum hardware is not yet there. Instead, we will explore the concepts, implement small‑scale examples using simulators, and discuss the roadmap to practical quantum advantage.

---

## **97.2 Quantum Computing Basics**

To understand QML, we need a basic grasp of quantum computing concepts.

### **97.2.1 Qubits and Superposition**

A classical bit is either 0 or 1. A **qubit** can be in a superposition of both states: 
|ψ⟩ = α|0⟩ + β|1⟩, where α and β are complex numbers and |α|² + |β|² = 1.

When measured, the qubit collapses to |0⟩ with probability |α|² and to |1⟩ with probability |β|².

### **97.2.2 Entanglement**

Entanglement is a quantum correlation between qubits. The state of one qubit cannot be described independently of the others. For example, the Bell state (|00⟩ + |11⟩)/√2: if you measure one qubit as 0, the other is guaranteed to be 0.

### **97.2.3 Quantum Gates**

Quantum gates manipulate qubits. Common gates:

- **Hadamard (H)**: Creates superposition.
- **Pauli-X, Y, Z**: Quantum analogues of NOT and rotations.
- **CNOT**: A two‑qubit gate that flips the target qubit if the control is |1⟩.

### **97.2.4 Quantum Circuits**

A quantum circuit is a sequence of gates applied to qubits, followed by measurement. This is the quantum analogue of a classical circuit.

### **97.2.5 Quantum Hardware Limitations**

Current quantum computers (NISQ – Noisy Intermediate‑Scale Quantum) have:

- **Limited qubits**: Typically 50‑100 qubits.
- **Noise**: Gate errors and decoherence limit circuit depth.
- **Short coherence times**: Qubits lose their quantum state quickly.

These limitations mean that only small, shallow circuits can run reliably. QML algorithms must be designed to be noise‑resilient.

---

## **97.3 Quantum Machine Learning Algorithms**

Several QML algorithms have been proposed. We'll review the most relevant.

### **97.3.1 Quantum Support Vector Machines (QSVM)**

Classical SVMs find a hyperplane that separates data. The kernel trick maps data to a higher‑dimensional space where separation is easier. Quantum computers can estimate kernels that are classically hard to compute.

The **QSVM** algorithm uses a quantum circuit to compute the kernel matrix. For a dataset {xᵢ}, the kernel entry K(xᵢ, xⱼ) = |⟨φ(xᵢ)|φ(xⱼ)⟩|², where |φ(x)⟩ is a quantum state prepared by a feature map circuit.

This kernel can be more expressive than classical kernels, potentially leading to better classification.

### **97.3.2 Variational Quantum Circuits (VQC)**

Variational quantum circuits are the quantum analogue of neural networks. They consist of:

1. A **data encoding** circuit that embeds classical data into quantum states.
2. A **variational** circuit with trainable parameters (rotation angles).
3. Measurement of observables to produce an output.

Training is done by a classical optimiser that adjusts the parameters to minimise a loss function. This hybrid approach (classical optimisation + quantum circuit) is the most promising for near‑term quantum advantage.

### **97.3.3 Quantum Neural Networks (QNN)**

QNNs extend VQCs with multiple layers and non‑linearities (though non‑linearity in quantum circuits is tricky; it often comes from measurement or repeated encoding).

### **97.3.4 Quantum Generative Models**

Quantum circuits can be trained to generate samples from probability distributions. This is the quantum analogue of generative adversarial networks (GANs) or variational autoencoders (VAEs). For time‑series, this could be used to generate realistic synthetic market data.

### **97.3.5 Quantum Optimisation for Training**

Training classical models often involves optimising a non‑convex loss function. Quantum optimisation algorithms (QAOA, quantum annealing) might find better minima faster, though this is still speculative.

---

## **97.4 Implementing a Simple Quantum Classifier with Qiskit**

Let's implement a small quantum classifier using Qiskit, IBM's quantum computing framework. We'll apply it to a synthetic time‑series classification problem: classifying whether a stock's return tomorrow will be positive or negative based on the last 3 days of returns.

**Note**: This example runs on a simulator. Running on real quantum hardware would require an IBM Quantum account and is subject to queue times and noise.

```python
# pip install qiskit qiskit-machine-learning qiskit-aer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Generate synthetic time‑series classification data
# Features: returns from last 3 days
# Target: sign of next day return (1 if positive, 0 otherwise)
np.random.seed(42)
n_samples = 200
n_features = 3

X, y = make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=42
)

# Scale features to [-1, 1] (suitable for quantum encoding)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = np.clip(X_scaled, -1, 1)  # clip to range

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)
```

### **97.4.1 Quantum Kernel Method**

First, we'll implement a quantum kernel estimator and use it with a classical SVM.

```python
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Set seed for reproducibility
algorithm_globals.random_seed = 42

# Define a feature map (encodes classical data into quantum state)
def create_feature_map(n_features):
    params = ParameterVector('x', length=n_features)
    qc = QuantumCircuit(n_features)
    
    # Encode data with angle encoding (Rx rotations)
    for i in range(n_features):
        qc.rx(params[i], i)
    
    # Add entanglement (CNOTs)
    for i in range(n_features - 1):
        qc.cx(i, i+1)
    
    return qc

feature_map = create_feature_map(n_features)

# Create quantum kernel
quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
    quantum_instance=Aer.get_backend('statevector_simulator')
)

# Train QSVC
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train, y_train)

# Evaluate
accuracy = qsvc.score(X_test, y_test)
print(f"QSVC test accuracy: {accuracy:.4f}")

# Compare with classical RBF SVM
from sklearn.svm import SVC
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
print(f"Classical SVM test accuracy: {svm.score(X_test, y_test):.4f}")
```

**Explanation**:

- We create a feature map circuit that encodes the 3 features using Rx rotations, then adds entanglement with CNOT gates.
- The `FidelityQuantumKernel` computes the kernel matrix by running this circuit for each pair of data points and measuring the overlap.
- `QSVC` is a quantum‑kernel version of a support vector classifier.
- On this small synthetic dataset, the quantum kernel may perform similarly to a classical RBF kernel. The true test is on problems where the quantum kernel provides an advantage.

### **97.4.2 Variational Quantum Classifier (VQC)**

Now, we'll implement a variational quantum classifier, which trains a parameterised quantum circuit.

```python
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import VQC

algorithm_globals.random_seed = 42

# Feature map (data encoding)
feature_map = ZZFeatureMap(feature_dimension=n_features, reps=2)

# Variational form (trainable circuit)
var_form = RealAmplitudes(n_features, reps=3)

# Optimizer
optimizer = COBYLA(maxiter=100)

# Create VQC
vqc = VQC(
    feature_map=feature_map,
    ansatz=var_form,
    optimizer=optimizer,
    quantum_instance=Aer.get_backend('statevector_simulator')
)

# Train
vqc.fit(X_train, y_train)

# Evaluate
accuracy = vqc.score(X_test, y_test)
print(f"VQC test accuracy: {accuracy:.4f}")

# Inspect the circuit
print(vqc.ansatz.draw())
```

**Explanation**:

- `ZZFeatureMap` encodes data using entangling ZZ interactions.
- `RealAmplitudes` is a common variational form with parameterised Ry rotations and CNOTs.
- The VQC trains these parameters to minimise the classification error.
- This hybrid approach (classical optimisation + quantum circuit) is the most practical for near‑term quantum hardware.

---

## **97.5 Applying QML to Time‑Series**

For the NEPSE system, we could imagine using QML for:

- **Classification of market regimes** (bull/bear/sideways) based on recent price and volume patterns.
- **Kernel methods** to find non‑linear relationships between stocks.
- **Quantum generative models** to generate realistic synthetic market scenarios for stress testing.
- **Optimisation** of portfolio allocation under constraints (a quantum‑amenable problem).

However, with current hardware, these are research projects. The example above is on a tiny dataset (3 features, 200 samples). Real NEPSE data has many more features (lags, technical indicators) and samples, far beyond what current quantum computers can handle.

---

## **97.6 Current Limitations and Challenges**

### **97.6.1 Hardware Constraints**
- **Qubit count**: Even the largest quantum computers (e.g., IBM's Osprey with 433 qubits) cannot encode the hundreds of features typical in time‑series prediction.
- **Noise**: Gate errors limit circuit depth. Our simple feature map and variational form may be too deep for current hardware.
- **Coherence time**: Qubits lose their state quickly, limiting the length of computations.

### **97.6.2 Encoding Classical Data**
Mapping classical data to quantum states is non‑trivial. Amplitude encoding (using 2ⁿ amplitudes to encode n features) is efficient but requires complex state preparation. Angle encoding (as we used) is simpler but uses one qubit per feature, which doesn't scale.

### **97.6.3 Readout and Measurement**
Extracting results from quantum computers is probabilistic; many shots are needed to get accurate estimates, adding overhead.

### **97.6.4 Classical Baselines**
For most time‑series tasks, classical models (XGBoost, LSTM) are extremely good and run on cheap, readily available hardware. Quantum advantage must be significant to justify the cost and complexity.

### **97.6.5 Algorithmic Maturity**
QML is a young field. Many proposed algorithms lack rigorous proof of advantage, and empirical results are on small, synthetic problems.

---

## **97.7 Potential Advantages (When Might Quantum Help?)**

Despite the challenges, there are scenarios where quantum could eventually shine:

- **Kernel methods** for very high‑dimensional feature spaces that are classically intractable.
- **Solving optimisation problems** that are part of training (e.g., finding global minima in non‑convex landscapes).
- **Sampling from complex probability distributions** for generative models.
- **Quantum neural networks** that exploit entanglement to represent correlations that classical networks cannot.

For time‑series, this might mean discovering hidden patterns that are exponentially hard to find classically.

---

## **97.8 Tools and Frameworks**

- **Qiskit**: IBM's open‑source framework for quantum computing. Includes `qiskit-machine-learning` for QML algorithms.
- **Pennylane**: A library for differentiable quantum programming, integrates with PyTorch and TensorFlow.
- **TensorFlow Quantum**: Google's library for hybrid quantum‑classical machine learning.
- **Cirq**: Google's quantum programming framework.

These tools allow you to design and simulate quantum circuits, and run them on real hardware (if you have access).

---

## **97.9 The Road Ahead**

Quantum computing is advancing rapidly. Milestones to watch:

- **Fault‑tolerant quantum computers**: With error correction, we could run much deeper circuits.
- **Increased qubit counts**: 1000+ qubit machines may be available in the next few years.
- **Quantum advantage demonstrations**: For specific problems, we may see clear speedups.

For time‑series prediction, the first practical applications may be in **finance** (portfolio optimisation, risk analysis) and **scientific computing** (simulating physical systems that generate time‑series).

---

## **97.10 Best Practices for Staying Informed**

1. **Follow key research groups**: IBM Quantum, Google Quantum AI, Xanadu, etc.
2. **Read review papers**: "Quantum Machine Learning: A Review" (Biamonte et al.) is a good starting point.
3. **Experiment with simulators**: Use Qiskit or Pennylane to get hands‑on experience.
4. **Monitor hardware progress**: Keep an eye on quantum volume, qubit counts, and error rates.
5. **Be sceptical**: Many claims of quantum advantage are overblown. Look for rigorous evidence.

---

## **Chapter Summary**

In this chapter, we ventured into the speculative but exciting world of quantum machine learning. We covered:

- The basics of quantum computing: qubits, superposition, entanglement, and gates.
- Key QML algorithms: quantum kernel methods, variational quantum circuits, and quantum neural networks.
- A practical implementation of a quantum classifier using Qiskit on a synthetic time‑series problem.
- The severe limitations of current quantum hardware for real‑world time‑series tasks.
- Potential future advantages and the importance of staying informed.

For now, your NEPSE prediction system should stick with classical methods. But by understanding QML, you are prepared for a future where quantum computers may offer a genuine edge. This field is evolving rapidly, and the techniques you've learned in this chapter will help you evaluate and adopt new quantum algorithms as they mature.

In the next chapter, we will discuss **Ethical AI and Responsible ML**, ensuring that our prediction systems are fair, transparent, and accountable.

---

**End of Chapter 97**